In [1]:
import pandas as pd
import numpy as np
import glob
import os
import re

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

## A- Clean kaggle data (only US and France):

In [3]:
kaggle_tweets = pd.read_csv('../data/tweets_us/kaggle_en_tweets/vaccination_all_tweets.csv')

In [4]:
kaggle_tweets.shape

(69718, 16)

In [5]:
kaggle_tweets.head(2)

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet
0,1340539111971516416,Rachel Roh,"La Crescenta-Montrose, CA","Aggregator of Asian American news; scanning diverse sources 24/7/365. RT's, Follows and 'Likes' will fuel me 👩‍💻",2009-04-08 17:52:46,405,1692,3247,False,2020-12-20 06:06:44,Same folks said daikon paste could treat a cytokine storm #PfizerBioNTech https://t.co/xeHhIMg1kF,['PfizerBioNTech'],Twitter for Android,0,0,False
1,1338158543359250433,Albert Fong,"San Francisco, CA","Marketing dude, tech geek, heavy metal & '80s music junkie. Fascinated by meteorology and all things in the cloud. Opinions are my own.",2009-09-21 15:27:30,834,666,178,False,2020-12-13 16:27:13,"While the world has been on the wrong side of history this year, hopefully, the biggest vaccination effort we've ev… https://t.co/dlCHrZjkhm",NaN,Twitter Web App,1,1,False


In [6]:
kaggle_tweets = kaggle_tweets.dropna(subset = ['user_location'])

### 1- Rename and drop columns:

In [7]:
kaggle_tweets = (kaggle_tweets
          .drop(columns = ['user_created', 'user_favourites', 'source', 'favorites', 'is_retweet'])
          .rename(columns = {'id': 'tweet_id'})
         )

### 2- Keep only tweets in France and USA:

In [8]:
# Get a list of user location (higher count on top)  
#list_cities = kaggle_tweets.groupby('user_location')['user_location'].count().sort_values(ascending=False).to_frame()

In [9]:
# Make lists of places of interest for the study (France, south and north of USA)
cities_fr = ['France', 'Paris, France', 'Nancy, France', 'Dammam, KSA | Tunisia | France', 'Alsace, France', 'Paris', 'La Belle Province', 'Bordeaux, France', 'Alpes-Maritimes, Provence-Alpe', '#CoronavirusMadeInPasteur', 'Rome, Italy - Beaulieu, France', 'Europe (usually Poland/France)', 'Paris, Limoges. France']
cities_us_north = ['La Crescenta-Montrose, CA', 'San Francisco, CA', 'Los Angeles, CA', 'Rochester, NY', 'Cranbury, NJ', 'USA🇺🇸', 'America', 'Medford, OR', 'LI, NYC, NJ, PA, NE, USA', 'Chicago, IL', 'Seattle, WA',  'New Jersey, USA', 'Oregon, USA', 'California, USA', 'New York, NY', 'Cambridge, MA', 'New Orleans, LA', 'Michigan', 'Philadelphia, Pa.', 'North America', 'Troy, Michigan;  Serial ✈️', 'Pennsylvania, USA', 'Saugerties, New York', 'Balboa Peninsula, SoCal USA', 'New York / New Jersey / Global', 'Minneapolis, MN', 'Vermont, US', 'Naperville, IL', 'Sacramento', 'New York, USA', 'Cherry Hill, NJ', 'NYC', 'San Jose, CA (USA)', 'Colorado Springs, Colorado ', 'Silicon Valley, CA', 'Washington, DC', 'D.C By Way Of BOSTON!🇳🇬', 'New York', 'Mequon, Wisconsin', 'Boston, MA', 'New York City, Westport, CT', 'Michigan, USA', 'US - IL - ID', 'Washington DC', 'CHICAGO!', 'NY', 'Brooklyn, NY', 'Los Angeles', 'Providence, RI', 'Boston', 'New York City', 'Rancho Cucamonga, CA', 'Syracuse, NY', 'New York & San Francisco', 'Philadelphia, PA', 'New Jersey', 'Buffalo, NY', 'Connecticut, USA', 'Rhode Island, USA', 'Northern Illinois', 'Rochester NY', 'Southern CA', 'Staten Island, New York', 'South Lake Tahoe, CA', 'Boston, MA.', 'washington', 'Paramus, NJ', 'Orange County, CA', 'Elmhurst, IL', 'Grand Rapids, MI', 'Long Beach, CA', 'Washington, D.C.', 'Northern California, USA', 'Princeton, NJ', 'Ventura, CA ', 'Washington D.C.', 'Rochester, MN', 'Denver, CO', 'Englewood, NJ', 'Katonah, NY', 'Chicago', 'Valencia,CA', 'Bellevue, WA', 'Minnesota, USA', 'Old Mystic, CT USA', 'San Francisco Bay Area', 'Santa Barbara, CA', 'Baltimore', 'Vermont, USA', 'Sunnyvale, CA', 'Bethesda, MD.', 'Queens, NY', 'Rochester MN', 'Social distancing in Manhattan', 'Morristown, NJ', 'Bronx, NY', 'DC Metro Area, USA', 'Maryland, USA', 'Stony Brook,  NY', 'Bryn Mawr, PA', 'Madison, WI', 'Coventry Rhode Island', 'Ann Arbor, ✋', 'Colorado, USA', 'Baltimore, MD', 'Pittsburgh, PA', 'Wisconsin, USA', 'minneapolis', 'Southern California', 'Northern California', 'Potomac  MD', 'Seaside, California, USA', 'Hartford, CT', 'Long Island   ', 'California', 'Paterson, NJ', 'NH', 'Manhattan, NYC', 'California Bay Area', 'VT', 'Stanford, CA', 'Duarte, CA', 'Los Angeles, California', 'Central California', 'Portland, ME', 'Philadelphia', 'Riverdale Park, MD ', 'Manhattan, NY', 'East Hampton, NY', 'Hollywood, CA, USA', 'South Cal', 'Ann Arbor, MI', 'Batavia, IL', 'Cambridge, MA, United States', 'Chicago-adjacent, IL', 'Metro North of Boston MA', 'New Haven, CT', 'Silver Spring, MD', 'West Hollywood, CA', 'So Cal', 'SoCal', 'California, United States', 'Sacramento, CA', 'Massachusetts, USA', 'Illinois, USA', 'Washington, USA', 'Fremont, CA, USA', 'New Jersey', 'Portland, OR', 'San Francisco', 'New York, New York', 'Santa Monica, CA', 'Connecticut', 'West Hartford, CT', 'Oakland, CA', 'Anaheim, CA', 'San Jose, CA', 'Buffalo, NY', 'Massachusetts', 'Los Angeles, California', 'Vermont, USA', 'Detroit, MI', 'Delaware, USA', 'Philadelphia', 'Boothbay Harbor, Maine', 'San Diego, California']
cities_us_south = ['Texas', 'Jefferson City, MO',  'LA | OC ', 'Beaumont, Texas', 'Kentucky', 'Dallas, TX', 'Lakeland, FL', 'Palm Beach, FL', 'Tampa, FL', 'Okolona, KY', 'Houston, TX', 'Miami, FL', 'McAllen, TX', 'Texas, USA', 'Miami', 'Charleston, SC', 'Florida, USA', 'West Virginia, USA', 'Raleigh, NC', 'Raleigh, NC, USA', 'South Carolina, USA', 'Durham, NC', 'Alabama', 'Cincinnati, OH ', '11150A Asheville Hwy Inman, SC', 'Tucson, AZ', 'Albuquerque, NM', 'Nashville, TN', 'St Petersburg, FL', 'Austin, TX', 'Kansas', 'Roanoke, VA', 'Clinton, SC', 'Miami Beach, FL', 'San Antonio, Texas', 'Atlanta, GA', 'Odessa, TX', 'Chapel Hill, NC', 'Scottsdale, AZ', 'NEVADA', 'Richmond, VA', 'Atlanta, Georgia', 'Tysons Corner, Virginia', 'Phoenix, AZ', 'Greensboro, NC', 'Las Vegas, NV', 'Harlingen, TX', 'North Carolina, USA', 'Indianapolis, IN', 'Spring, TX', 'Idaho, USA', 'Athens, GA', 'Jacksonville, FL', 'Houston Texas Based', 'Jackson, MS', 'Covid Hell, AL', 'Kansas City, KS', 'Fort Worth', 'Charlottesville, VA', 'Kansas City, MO', 'Florida', 'Dayton, OH', 'Saint Louis, MO, USA', 'St Louis, MO', 'Knoxville, TN', 'Birmingham, AL', 'Columbia, MO', 'Morgantown, WV', 'TX', 'Allen, TX', 'Arkansas', 'Gainesville, FL', 'Smithville, TX', 'Augusta, GA', 'West Texas', 'Oklahoma City & Tulsa, OK', 'Oklahoma, USA', 'Gainesville, FL, USA', 'Arkansas, United States', 'Sugar Land, TX', 'Miami, Fl', 'Central FL', 'arkansas', 'Riverdale, UT', 'Nashville', 'Dallas', 'Houston', 'Reno, NV', 'Tampa Bay, FL', 'Charlotte, NC', 'Austin', 'Kansas City ', 'South Florida', 'Salt Lake City, UT', 'Alabama, USA', 'Ohio, USA', 'Virginia, USA', 'Orlando, FL', 'Las Vegas', 'Lufkin, TX', 'Arizona, USA', 'Cleveland, OH', 'Georgia, USA', 'Fort Lauderdale, FL', 'New York, Sarasota, FL', 'Columbus, OH', 'Louisville, KY', 'Tennessee, USA', 'Missouri, USA', 'Indiana, USA', 'Kentucky, USA', 'Oklahoma City, OK', 'North Carolina', 'New Mexico, USA', 'Rio Grande Valley, TX', 'Houston, Tx', 'Ohio', 'Cleveland, Ohio', 'Utah, USA', 'Indiana', 'Salt Lake City, Utah', 'Dallas, Texas', 'South Carolina', 'Columbus, Ohio', 'Corpus Christi, TX', 'Tampa, FL - United States', 'Atlanta', 'Indianapolis', 'Salt Lake City, UT', 'Alabama', 'TX', 'Fort Worth, TX', 'Jacksonville, N.C.', 'Tennessee', 'Arkansas, USA', 'Baton Rouge, LA', 'Arkansas', 'Jupiter, FL', 'Louisiana, USA', 'Tulsa, OK']

In [10]:
# Make dfs with the list of cities
tweets_fr = kaggle_tweets[kaggle_tweets['user_location'].isin(cities_fr)]
tweets_us_north = kaggle_tweets[kaggle_tweets['user_location'].isin(cities_us_north)]
tweets_us_south = kaggle_tweets[kaggle_tweets['user_location'].isin(cities_us_south)]

## B- Clean data from Tweepy:

### 1- France:

**Merge files to one file and add a column with the respective vaccines names(week 2-3):**

In [11]:
# merging astrazeneca files
astra_fr = pd.concat(
    map(pd.read_csv, ['../data/tweets_fr/astra/tweets_fr_tweets_fr_aztra_2021_05_18_to_05_12.csv', '../data/tweets_fr/astra/tweets_fr_tweets_fr_astra_2021_05_19_to_05_13.csv']), ignore_index=True)
astra_fr.shape

(667, 9)

In [12]:
# Make a new column with vaccine name
astra_fr['vaccine'] = 'astrazeneca'

In [13]:
# merging J&J files
jandj_fr = pd.concat(
    map(pd.read_csv, ['../data/tweets_fr/JandJ/tweets_fr_jandj_2021_05_19_to_05_14.csv', '../data/tweets_fr/JandJ/tweets_fr_jandj_2021_05_18_to_05_11.csv']), ignore_index=True)
jandj_fr.shape

(69, 9)

In [14]:
# Make a new column with vaccine name
jandj_fr['vaccine'] = 'johnson'

In [15]:
# merging pfizer files
pfizer_fr = pd.concat(
    map(pd.read_csv, ['../data/tweets_fr/pfizer/tweets_fr_pfizer_2021_05_14_to_05_11.csv', '../data/tweets_fr/pfizer/tweets_fr_pfizer_2021_05_18_to_05_15.csv']), ignore_index=True)
pfizer_fr.shape

(2577, 8)

In [16]:
# Make a new column with vaccine name
pfizer_fr['vaccine'] = 'pfizer'

In [17]:
# moderna
moderna_fr = pd.read_csv('../data/tweets_fr/moderna/tweets_fr_moderna_2021_05_18_to_05_11.csv')
moderna_fr['vaccine'] = 'moderna'

In [18]:
# Concatenate dataframes together
vaccines_fr = pd.concat([astra_fr, jandj_fr, pfizer_fr, moderna_fr], ignore_index=True)

In [19]:
vaccines_fr.shape

(4111, 10)

In [20]:
# Get tweets with a different search
all_fr = pd.read_csv('../data/tweets_fr/tweets_fr_all_2021_05_19_to_05_11.csv')

In [21]:
all_fr.shape

(5099, 9)

In [22]:
#columns = ["Tweet Id", "Tweet Date", "location", "Follower Count", "Account Verified", "Favorite Count", "Retweets", "Tweet Text", "hashtags"]

In [23]:
# Get the tweets ID that are in common with the 2 different searchs
common = all_fr.merge(vaccines_fr, how='inner', on=["Tweet Id"]) 

In [24]:
common.shape

(3206, 18)

In [25]:
# Get the tweets that are not in vaccine_fr
all_fr_no_com = all_fr[(~all_fr["Tweet Id"].isin(common["Tweet Id"]))]
all_fr_no_com.shape

(1907, 9)

In [27]:
# Add a column with the respective vaccine for each tweet
all_fr_no_com.loc[all_fr_no_com["Tweet Text"].str.contains("pfizer|Pfizer"), "vaccine"] = "pfizer"
all_fr_no_com.loc[all_fr_no_com["Tweet Text"].str.contains("moderna|Moderna"), "vaccine"] = "moderna"
all_fr_no_com.loc[all_fr_no_com["Tweet Text"].str.contains("astrazeneca|Aztrazeneca"), "vaccine"] = "astrazeneca"
all_fr_no_com.loc[all_fr_no_com["Tweet Text"].str.contains("johnson|Johnson"), "vaccine"] = "johnson"

In [28]:
# Drop rows with NaN values in column "vaccine"
all_fr_no_com = all_fr_no_com.dropna(subset=[9], axis=1)

In [30]:
# Concatenate all_fr_no_com to vaccines_fr
vaccines_fr = pd.concat([vaccines_fr, all_fr_no_com], ignore_index=True)

In [31]:
# Rename and drop columns 
vaccines_fr = (vaccines_fr
          .drop(columns = ['Account Verified', 'Favorite Count', 'hashtags'])
          .rename(columns = {'Tweet Id': 'id', 'Tweet Date': 'date', 'Follower Count': 'follower_count', 'Retweets': 'retweets', 'Tweet Text': 'text'})
         )

In [32]:
# change "date" object type to datetime
vaccines_fr['date'] = vaccines_fr['date'].astype('Datetime64')

**Encode 'utf-8' characters with real characters:**

In [33]:
# use eval to decode "utf-8" and remove "\n"
vaccines_fr["text"] = vaccines_fr["text"].map(lambda x: eval(x).decode("utf-8").replace('\n', ' '))

**Removing Twitter Handles (@user) and links:**

In [34]:
def remove_pattern(input_text, pattern):
    reg = re.findall(pattern, input_text)
    for x in reg:
        input_text = re.sub(x, '', input_text)
        
    return input_text

In [35]:
# twitter handles
vaccines_fr["text"] = np.vectorize(remove_pattern)(vaccines_fr['text'], "@[\w]*")

In [36]:
# links
vaccines_fr["text"] = np.vectorize(remove_pattern)(vaccines_fr['text'], "https://[\w\.\\/]*")

**Save "vaccine_fr" as a csv file:**

In [37]:
vaccines_fr.to_csv('../data/vaccines_fr.csv', index=False)

**Merge files to one file and add a column with the respective vaccines names(week 4):**

In [39]:
pfizer_fr_week4 = pd.read_csv('../data/tweets_fr/pfizer/tweets_fr_pfizer_2021_05_29_to_05_22.csv')
pfizer_fr_week4['vaccine'] = 'pfizer'
moderna_fr_week4 = pd.read_csv('../data/tweets_fr/moderna/tweets_fr_moderna_2021_05_29_to_05_22.csv')
moderna_fr_week4['vaccine'] = 'moderna'
astra_fr_week4 = pd.read_csv('../data/tweets_fr/astra/tweets_fr_astra_2021_05_29_to_05_22.csv')
astra_fr_week4['vaccine'] = 'astrazeneca'
janssen_fr_week4 = pd.read_csv('../data/tweets_fr/JandJ/tweets_fr_jandj_2021_05_29_to_05_22.csv')
janssen_fr_week4['vaccine'] = 'johnson'

In [40]:
# Concatenate all_fr_no_com to vaccines_fr
vaccines_fr_week4 = pd.concat([pfizer_fr_week4, moderna_fr_week4, astra_fr_week4, janssen_fr_week4], ignore_index=True)

In [42]:
# Rename and drop columns 
vaccines_fr_week4 = (vaccines_fr_week4
          .drop(columns = ['username', 'acctdesc', 'Friends Count','Account Verified', 'Favorite Count', 'hashtags'])
          .rename(columns = {'Tweet Id': 'id', 'Tweet Date': 'date', 'Follower Count': 'follower_count', 'Retweets': 'retweets', 'Tweet Text': 'text'})
         )

In [44]:
# change "date" object type to datetime
vaccines_fr_week4['date'] = vaccines_fr_week4['date'].astype('Datetime64')

In [46]:
# use eval to decode "utf-8" and remove "\n"
vaccines_fr_week4["text"] = vaccines_fr_week4["text"].map(lambda x: eval(x).decode("utf-8").replace('\n', ' '))
# remove twitter handles
vaccines_fr_week4["text"] = np.vectorize(remove_pattern)(vaccines_fr_week4['text'], "@[\w]*")
# remove links
vaccines_fr_week4["text"] = np.vectorize(remove_pattern)(vaccines_fr_week4['text'], "https://[\w\.\\/]*")

In [51]:
#Save file
vaccines_fr_week4.to_csv('../data/vaccines_fr_week4.csv', index=False)

### 2- USA:

**NORTHERN STATES:**

In [53]:
# Concat two csv files for week 3 and 4 of May
pfizer_north = pd.concat(
    map(pd.read_csv, ['../data/tweets_us/us_north/pfizer/tweets_us_north_pfizer_2021_05_22_to_05_16.csv', '../data/tweets_us/us_north/pfizer/tweets_us_north_pfizer_2021_05_31_to_05_23.csv']), ignore_index=True)
# Make a new column with vaccine name
pfizer_north['vaccine'] = 'pfizer'

In [54]:
# Concat two csv files for week 3 and 4 of May
moderna_north = pd.concat(
    map(pd.read_csv, ['../data/tweets_us/us_north/moderna/tweets_us_north_moderna_2021_05_22_to_05_16.csv', '../data/tweets_us/us_north/moderna/tweets_us_north_moderna_2021_05_31_to_05_23.csv']), ignore_index=True)
# Make a new column with vaccine name
moderna_north['vaccine'] = 'moderna'

In [55]:
# Concat two csv files for week 3 and 4 of May
janssen_north = pd.concat(
    map(pd.read_csv, ['../data/tweets_us/us_north/JandJ/tweets_us_north_jandj_2021_05_22_to_05_16.csv', '../data/tweets_us/us_north/JandJ/tweets_us_north_jandj_2021_05_31_to_05_23.csv']), ignore_index=True)
# Make a new column with vaccine name
janssen_north['vaccine'] = 'johnson'

In [56]:
# Concat two csv files for week 3 and 4 of May
astra_north = pd.concat(
    map(pd.read_csv, ['../data/tweets_us/us_north/astra/tweets_us_north_astra_2021_05_22_to_05_16.csv', '../data/tweets_us/us_north/astra/tweets_us_north_astra_2021_05_31_to_05_23.csv']), ignore_index=True)
# Make a new column with vaccine name
astra_north['vaccine'] = 'astrazeneca'

In [58]:
# Concatenate dataframes together
tweets_us = pd.concat([astra_north, janssen_north, pfizer_north, moderna_north], ignore_index=True)

In [60]:
# Concat two csv files for week 3 and 4 of May
all_north = pd.concat(
    map(pd.read_csv, ['../data/tweets_us/us_north/tweets_us_north_all_2021_05_22_to_05_16.csv', '../data/tweets_us/us_north/tweets_us_north_all_2021_05_31_to_05_23.csv']), ignore_index=True)

In [61]:
# Get the tweets ID that are in common with the 2 different searchs
common_us_n = all_north.merge(tweets_us, how='inner', on=["Tweet Id"])
common_us_n.shape

(15682, 24)

In [62]:
# Get the tweets that are not in tweets_us
all_us_n_no_com = all_fr[(~all_north["Tweet Id"].isin(common["Tweet Id"]))]
all_us_n_no_com.shape

/Users/armelleleguelte/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


(5099, 9)

In [63]:
# Add a column with the respective vaccine for each tweet
all_us_n_no_com.loc[all_us_n_no_com["Tweet Text"].str.contains("pfizer|Pfizer"), "vaccine"] = "pfizer"
all_us_n_no_com.loc[all_us_n_no_com["Tweet Text"].str.contains("moderna|Moderna"), "vaccine"] = "moderna"
all_us_n_no_com.loc[all_us_n_no_com["Tweet Text"].str.contains("astrazeneca|Aztrazeneca"), "vaccine"] = "astrazeneca"
all_us_n_no_com.loc[all_us_n_no_com["Tweet Text"].str.contains("johnson|Johnson"), "vaccine"] = "johnson"

In [67]:
all_us_n_no_com = all_us_n_no_com.dropna(subset=['vaccine'])

In [72]:
# Concatenate all_fr_no_com to vaccines_fr
tweets_us_north = pd.concat([tweets_us, all_us_n_no_com], ignore_index=True)

In [73]:
# Rename and drop columns 
tweets_us_north = (tweets_us_north
          .drop(columns = ['username', 'acctdesc', 'Friends Count','Account Verified', 'Favorite Count', 'hashtags'])
          .rename(columns = {'Tweet Id': 'id', 'Tweet Date': 'date', 'Follower Count': 'follower_count', 'Retweets': 'retweets', 'Tweet Text': 'text'})
         )

In [75]:
# change "date" object type to datetime
tweets_us_north['date'] = tweets_us_north['date'].astype('Datetime64')

In [76]:
# use eval to decode "utf-8" and remove "\n"
tweets_us_north["text"] = tweets_us_north["text"].map(lambda x: eval(x).decode("utf-8").replace('\n', ' '))
# remove twitter handles
tweets_us_north["text"] = np.vectorize(remove_pattern)(tweets_us_north['text'], "@[\w]*")
# remove links
tweets_us_north["text"] = np.vectorize(remove_pattern)(tweets_us_north['text'], "https://[\w\.\\/]*")

In [80]:
#Save file
#tweets_us_north.to_csv('../data/tweets_us_north.csv', index=False)

**SOUTHERN STATES:**

In [101]:
# merging the files pfizer
joined_files = os.path.join("../data/tweets_us/us_south/pfizer/", "tweets_us*.csv")
  
# A list of all joined files is returned
joined_list = glob.glob(joined_files)
  
# Finally, the files are joined
pfizer_south = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)

# Make a new column with vaccine name
pfizer_south['vaccine'] = 'pfizer'

In [102]:
# merging the files moderna
joined_files = os.path.join("../data/tweets_us/us_south/moderna/", "tweets_us*.csv")
  
# A list of all joined files is returned
joined_list = glob.glob(joined_files)
  
# Finally, the files are joined
moderna_south = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)

# Make a new column with vaccine name
moderna_south['vaccine'] = 'moderna'

In [103]:
# merging the files janssen
joined_files = os.path.join("../data/tweets_us/us_south/JandJ/", "tweets_us*.csv")
  
# A list of all joined files is returned
joined_list = glob.glob(joined_files)
  
# Finally, the files are joined
janssen_south = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)

# Make a new column with vaccine name
janssen_south['vaccine'] = 'johnson'

In [104]:
# merging the files astra
joined_files = os.path.join("../data/tweets_us/us_south/astra/", "tweets_us*.csv")
  
# A list of all joined files is returned
joined_list = glob.glob(joined_files)
  
# Finally, the files are joined
astra_south = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)

# Make a new column with vaccine name
astra_south['vaccine'] = 'astrazeneca'

In [105]:
# Concatenate dataframes together
tweets_us_south = pd.concat([astra_north, janssen_north, pfizer_north, moderna_north], ignore_index=True)

In [106]:
# Concat two csv files for week 3 and 4 of May
all_south = pd.concat(
    map(pd.read_csv, ['../data/tweets_us/us_south/tweets_us_all_south_2021_05_22_to_05_16.csv', '../data/tweets_us/us_south/tweets_us_all_south_2021_05_31_to_05_23.csv']), ignore_index=True)

In [110]:
# at least one tweet was scraped wrongly and the values are not in the correct rows, I decide to drop it
all_south = all_south[all_south["Tweet Id"] != 'He/Him']

In [119]:
# drop columns with no text ( probably other tweets not scrapped properly)
all_south = all_south.dropna(subset=['Tweet Text'])

In [121]:
# Add a column with the respective vaccine for each tweet
all_south.loc[all_south["Tweet Text"].str.contains("pfizer|Pfizer"), "vaccine"] = "pfizer"
all_south.loc[all_south["Tweet Text"].str.contains("moderna|Moderna"), "vaccine"] = "moderna"
all_south.loc[all_south["Tweet Text"].str.contains("astrazeneca|Aztrazeneca"), "vaccine"] = "astrazeneca"
all_south.loc[all_south["Tweet Text"].str.contains("johnson|Johnson"), "vaccine"] = "johnson"

/Users/armelleleguelte/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/Users/armelleleguelte/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [122]:
all_south = all_south.dropna(subset=['vaccine'])

In [124]:
# Concatenate all_fr_no_com to vaccines_fr
tweets_us_south = pd.concat([tweets_us_south, all_south], ignore_index=True)

In [125]:
# Rename and drop columns 
tweets_us_south = (tweets_us_south
          .drop(columns = ['username', 'acctdesc', 'Friends Count','Account Verified', 'Favorite Count', 'hashtags'])
          .rename(columns = {'Tweet Id': 'id', 'Tweet Date': 'date', 'Follower Count': 'follower_count', 'Retweets': 'retweets', 'Tweet Text': 'text'})
         )

In [128]:
# change data types
tweets_us_south['date'] = tweets_us_south['date'].astype('Datetime64')
tweets_us_south['id'] = tweets_us_south['id'].astype('int64')
tweets_us_south['follower_count'] = tweets_us_south['follower_count'].astype('int64')

In [129]:
# use eval to decode "utf-8" and remove "\n"
tweets_us_south["text"] = tweets_us_south["text"].map(lambda x: eval(x).decode("utf-8").replace('\n', ' '))
# remove twitter handles
tweets_us_south["text"] = np.vectorize(remove_pattern)(tweets_us_south['text'], "@[\w]*")
# remove links
tweets_us_south["text"] = np.vectorize(remove_pattern)(tweets_us_south['text'], "https://[\w\.\\/]*")

In [134]:
#Save file
tweets_us_south.to_csv('../data/tweets_us_south.csv', index=False)